# Предсказание погоды
## Coding Stream 2020.11.28

Что мы хотим сделать?  
У нас есть данные по погоде за несколько лет со многих метеостанций.  
Мы попытаемся предсказать погоду на следующий день по предыдущим N дням с помощью линейной регрессии.

Для начала качаем данные с ftp.

In [ ]:
import shutil
import urllib
from contextlib import closing

with closing(urllib.request.urlopen('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2018.csv.gz')) as r:
    with open('./2018.csv.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

Файлы довольно большие, поэтому мы не хотим их считывать с помощью pandas. Вместо этого мы используем средства самого Python.

Формат следующий. Файл csv, каждая строка такая:   
<id_станции>, <дата>, <метрика>, <значение>, <другие_параметры>  
Нас интересуют первые 4 столбца

Мы прочитаем все интересующие нас данные в память и выберем те метеостанции, которые вернули показания по средней дневной температуре и направлению ветра в каждый из дней года.

Dask – альтернатива pandas, позволяет делать многопоточные вычисления.

In [22]:
import sys
from sklearn import linear_model


def days_in_year(year):
    if year % 4 != 0:
        return 365
    if year % 400 == 0:
        return 366
    if year % 100 == 0:
        return 365
    return 366


def parse_years(years):
    """
    codes = {
        code: {
            'dt': {
                'TAVG': float,
                'SNWD': float
            }
        }
    }
    
    result = {
        code: [
            (temp1, wind1),
            (temp2, wind2),
            ...
        ]
    }
    """
    result = {}
    for year in years:
        codes = {}
        for line in open('{year}.csv'.format(year=year), 'rt'):
            if line and line[-1] == '\n':
                line = line[:-1]
            data = line.split(',')
            code, dt, metric, value = data[0:4]
            if metric not in ['TAVG', 'SNWD']:
                continue
            if code not in codes:
                codes[code] = {}
            dates = codes[code]
            if dt not in dates:
                dates[dt] = {}
            dates[dt][metric] = float(value) / 10
        for code, dates in codes.items():
            data_temp = [dates[dt]['TAVG'] for dt in sorted(dates.keys()) if 'TAVG' in dates[dt]]
            data_wind = [dates[dt]['SNWD'] for dt in sorted(dates.keys()) if 'SNWD' in dates[dt]]
            if len(data_temp) != days_in_year(year) or len(data_wind) != days_in_year(year):
                continue
            if code not in result:
                result[code] = list(zip(data_temp, data_wind))
            else:
                result[code] += list(zip(data_temp, data_wind))
    return result

In [ ]:
for i in generator:
    i...

In [65]:
!tail -10 '2016.csv'

ZA000067743,20161231,TAVG,272,H,,S,
ZI000067775,20161231,TMAX,265,,,S,
ZI000067775,20161231,PRCP,20,,,S,
ZI000067775,20161231,TAVG,211,H,,S,
ZI000067975,20161231,TMIN,190,,,S,
ZI000067975,20161231,PRCP,109,,,S,
ZI000067975,20161231,TAVG,221,H,,S,
ZI000067983,20161231,TMAX,258,,,S,
ZI000067983,20161231,PRCP,0,,,S,
ZI000067983,20161231,TAVG,234,H,,S,


In [64]:
!ls -lh

total 4867760
-rw-r--r--  1 lemhell  staff   1,2G 28 ноя 02:32 2016.csv
-rw-r--r--  1 lemhell  staff   1,1G 28 ноя 02:38 2017.csv
-rw-r--r--  1 lemhell  staff    17K 28 ноя 14:32 2020.11.28 Coding Stream.ipynb
-rw-------@ 1 lemhell  staff   2,9K  9 июл 11:09 readme.txt


In [23]:
def iterate_data(data, window):
    """
    Сюда приходит result
    
    yield создает генератор, который при каждом вызове (или итерации по нему) возвращает один объект за раз
    Это полезно, когда у нас большие файлы, которые мы не хотим грузить в память
    """
    for _, items in data.items():
        for i in range(0, len(items) - window - 1):
             yield items[i : i + window], items[i + window][0]

def calc_mse(data, window, model):
    result, count = 0.0, 0
    for x, y in iterate_data(data, window):
        result += ((model(x) - y)**2)
        count += 1
    return result / count


def prepare_features(x):
    return [x[i][0] for i in range(len(x))] + [x[i][1] for i in range(len(x))]

In [24]:
def train_linear_regression(data, window):
    X, Y = [], []
    for x, y in iterate_data(data, window):
        X.append(prepare_features(x))
        Y.append(y)
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)
    return regr


def apply_regression(data, model):
    return model.predict([prepare_features(data)])[0]


def baseline_model(data):
    # data is a list of tuples (temp, wind)
    return data[-1][0]

Прочитаем наши данные  
Данные для обучения и валидации не должны пересекаться – иначе это прямой путь к переобучению

In [25]:
train = parse_years([2016])
test = parse_years([2017])

Зададим окно, с которым мы будем обучать модель. Это будет количество дней (признаков), на которые мы будем смотреть

In [26]:
window = 14

Посчитаем метрики на обучении и валидации для нашей baseline-модели.  
В качестве базовой модели выберем такую - будем всегда возвращать температуру за вчерашний день.  

Почему мы делаем так: наша задача состоит в том, чтобы предсказать температуру на следующий день.  
В качестве такого предсказания мы будем возвращать температуру за вчерашний день, так как обычно изменение температуры за соседние дни бывает не очень сильное.

In [28]:
calc_mse(train, window, baseline_model)

8.72859275309986

In [29]:
calc_mse(test, window, baseline_model)

8.853850137207615

Давайте обучим нашу модель линейной регрессии:

In [30]:
model = train_linear_regression(train, window)

И посмотрим на результаты:

In [31]:
calc_mse(train, window, lambda x: apply_regression(x, model))

7.317883214892826

In [32]:
calc_mse(test, window, lambda x: apply_regression(x, model))

7.746158385484083

Получилось, что линейная регрессия справляется лучше, чем baseline-модель.  
Также, метрики не сильно отличаются между обучением и валидацией. Это значит, что переобучения не произошло.

Что, если мы возьмем другой размер окна?

In [40]:
window = 30
model = train_linear_regression(train, window)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 7.198350517415528
MSE test 7.697311945722793


Метрика на валидации заметно выше, чем на обучении. Похоже, мы встретились с переобучением.  
Попробуем применить регуляризацию:

In [43]:
from sklearn.linear_model import Ridge

def train_ridge_regression(data, window, alpha=1.0):
    X, Y = [], []
    for x, y in iterate_data(data, window):
        X.append(prepare_features(x))
        Y.append(y)
    regr = Ridge(alpha=alpha)
    regr.fit(X, Y)
    return regr

In [44]:
window = 30
model = train_ridge_regression(train, window, alpha=10)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 7.198350517690733
MSE test 7.697312941918887


In [57]:
window = 7
model = train_ridge_regression(train, window, alpha=100)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 7.534483237026629
MSE test 7.908413797271512


Давайте попробуем случайный лес (осторожно, обучается намного дольше регрессии):

In [49]:
from sklearn.ensemble import RandomForestRegressor

def train_random_forest_regression(data, window, n_estimators=10, max_depth=None):
    X, Y = [], []
    for x, y in iterate_data(data, window):
        X.append(prepare_features(x))
        Y.append(y)
    regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    regr.fit(X, Y)
    return regr

In [48]:
window = 30
model = train_random_forest_regression(train, window)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 0.7499780430456969
MSE test 9.17987124192312


Видим, что модель очень сильно переобучилась. Попробуем ограничить глубину деревьев:

In [50]:
window = 30
model = train_random_forest_regression(train, window, max_depth=2)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 17.667707825236793
MSE test 17.162301206039377


Переобучения больше нет, но результаты даже намного хуже, чем у baseline-модели. По-хорошему тут нужно запускать grid search на разные параметры, но это очень долго.

Над чем еще можно было бы поработать?

 * Использовать те метеостанции, которые не вернули данные за весь год - улучшит это обучение или ухудшит?
 * Учесть месяц/время года в качестве фичей
 * Учесть геопозицию метеостанции (это есть в соседних файлах на ftp, их можно скачать)
 * Что еще?